In [49]:
import pandas as pd
import numpy as np

# IMU Changes
- IMU data was recorded at 200hz.
- Because of the high frequency of this data, adding deltas (between 1000 and 1100 microseconds) were not significant to cause Kimera-VIO to crash.
- Will now modify raw data and change it to 100 hz.

In [106]:
df = pd.read_csv('./mav0/imu0/data.csv')

In [107]:
df['#timestamp [ns]'] = df['#timestamp [ns]'].astype('float64')

In [108]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22800 entries, 0 to 22799
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   #timestamp [ns]      22800 non-null  float64
 1   w_RS_S_x [rad s^-1]  22800 non-null  float64
 2   w_RS_S_y [rad s^-1]  22800 non-null  float64
 3   w_RS_S_z [rad s^-1]  22800 non-null  float64
 4   a_RS_S_x [m s^-2]    22800 non-null  float64
 5   a_RS_S_y [m s^-2]    22800 non-null  float64
 6   a_RS_S_z [m s^-2]    22800 non-null  float64
dtypes: float64(7)
memory usage: 1.2 MB


In [109]:
# Check the deltas.
orig_deltas = np.unique(np.diff(df['#timestamp [ns]']), return_counts=True)

In [110]:
original_rate = 200
reduction_factor = 2
print("The frequency will be reduced to:", original_rate/reduction_factor)

The frequency will be reduced to: 100.0


In [111]:
# Reducing frequency
df = df.iloc[::reduction_factor]

In [112]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11400 entries, 0 to 22798
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   #timestamp [ns]      11400 non-null  float64
 1   w_RS_S_x [rad s^-1]  11400 non-null  float64
 2   w_RS_S_y [rad s^-1]  11400 non-null  float64
 3   w_RS_S_z [rad s^-1]  11400 non-null  float64
 4   a_RS_S_x [m s^-2]    11400 non-null  float64
 5   a_RS_S_y [m s^-2]    11400 non-null  float64
 6   a_RS_S_z [m s^-2]    11400 non-null  float64
dtypes: float64(7)
memory usage: 623.6 KB


In [113]:
# Check deltas after downsampling.
new_deltas = np.unique(np.diff(df['#timestamp [ns]']), return_counts=True)

In [114]:
#Since we have reduced the frequency, we should get a fractional value when we divide the original deltas by the new deltas
orig_deltas[0] / new_deltas[0]

array([0.5      , 0.5000128])

In [115]:
df.head(10)

,#timestamp [ns],w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]
0,1.413393e+18,0.094946,0.020246,0.058643,9.128357,0.106239,-2.606934
2,1.413393e+18,-0.011170,0.027925,0.069813,9.398040,1.618097,-2.705001
4,1.413393e+18,-0.092153,0.036303,0.078889,8.989429,-0.130755,-2.713173
6,1.413393e+18,-0.005585,0.009076,0.113795,9.299973,-1.234003,-2.476179
8,1.413393e+18,0.061436,0.023736,0.091455,9.275456,0.073550,-2.582418
10,1.413393e+18,0.009774,0.028623,0.060039,9.471589,0.400438,-2.606934
12,1.413393e+18,-0.032114,0.042586,0.086568,9.332662,0.122583,-2.705001
14,1.413393e+18,-0.030718,0.041888,0.094946,9.659550,-0.375922,-2.917478
16,1.413393e+18,-0.000698,0.024435,0.077493,9.291801,-0.490333,-2.598762
18,1.413393e+18,0.032812,0.008378,0.075398,9.447073,-0.098067,-2.655968


In [116]:
max_nano_seconds_to_add = max(np.unique(np.diff(df['#timestamp [ns]'])))

In [117]:
max_nano_seconds_to_add

10000128.0

In [131]:
(max_nano_seconds_to_add - 1)/2

5000063.5

### Introducing Minor Variation in IMU Timestamps

- 1,000,000 ns = 1 ms
- 10,000,000 ns = 10 ms (current delta)
- We are introducing at most 5,000,000 ns (5ms) jitter.
- In a worst case scenario, we can have 5,00,000 ns being subtracted and then for the next data point, we we can subtract none, leaving a max delta of 15,000,000 ns = 15 ms.
- This means that in terms of 'jitter', we are introducing +/- 5ms.
- If we introduce jitter of more than 10ms, we will have instances where timestamps overlap!

In [132]:
random_nanoseconds_to_add = np.random.uniform(low=0, high=(max_nano_seconds_to_add - 1)/2, size=10)

In [133]:
random_nanoseconds_to_add

array([1178065.67618299,  975229.3783837 , 4707068.63388826,
       3595286.29665765,  687120.18172398, 3069300.31068198,
        343318.34528093, 2943023.99132026, 3517968.26211942,
       1887533.67558715])

In [134]:
default_ts_arr = df['#timestamp [ns]'].values

In [135]:
np.unique(np.diff(default_ts_arr))

array([ 9999872., 10000128.])

In [136]:
modified_ts_arr = np.array([])

In [137]:
for def_ts in default_ts_arr:
    modified_ts = float(def_ts - np.random.choice(random_nanoseconds_to_add, 1)[0])
    modified_ts_arr = np.append(modified_ts_arr, modified_ts)

In [143]:
np.unique(np.diff(modified_ts_arr))

array([ 5636096.,  5636352.,  5979904.,  5980160.,  6267904.,  6268160.,
        6470912.,  6471168.,  6747904.,  6748160.,  6825216.,  6825472.,
        7091712.,  7091968.,  7169024.,  7169280.,  7180288.,  7180544.,
        7273984.,  7274240.,  7379712.,  7379968.,  7400192.,  7400448.,
        7457024.,  7457280.,  7582720.,  7582976.,  7617792.,  7618048.,
        7660032.,  7660288.,  7744000.,  7744256.,  7905792.,  7906048.,
        8032000.,  8032256.,  8108800.,  8109056.,  8235008.,  8235264.,
        8235776.,  8236032.,  8292096.,  8292352.,  8361984.,  8362240.,
        8369408.,  8369664.,  8455680.,  8455936.,  8799488.,  8799744.,
        8810752.,  8811008.,  8818176.,  8818432.,  8888064.,  8888320.,
        8944384.,  8944640.,  9087488.,  9087744.,  9165056.,  9165312.,
        9290496.,  9290752.,  9347584.,  9347840.,  9368064.,  9368320.,
        9424896.,  9425152.,  9473792.,  9474048.,  9508864.,  9509120.,
        9551104.,  9551360.,  9656064.,  9656320., 

In [139]:
np.unique(np.diff(modified_ts_arr)) > max_nano_seconds_to_add * 1.5

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [144]:
df['#timestamp [ns]'] = modified_ts

In [145]:
np.unique(np.diff(df['#timestamp [ns]']), return_counts=True)

(array([0.]), array([11399]))

### End Section

In [146]:
df.head(10)

,#timestamp [ns],w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]
0,1.413393e+18,0.094946,0.020246,0.058643,9.128357,0.106239,-2.606934
2,1.413393e+18,-0.011170,0.027925,0.069813,9.398040,1.618097,-2.705001
4,1.413393e+18,-0.092153,0.036303,0.078889,8.989429,-0.130755,-2.713173
6,1.413393e+18,-0.005585,0.009076,0.113795,9.299973,-1.234003,-2.476179
8,1.413393e+18,0.061436,0.023736,0.091455,9.275456,0.073550,-2.582418
10,1.413393e+18,0.009774,0.028623,0.060039,9.471589,0.400438,-2.606934
12,1.413393e+18,-0.032114,0.042586,0.086568,9.332662,0.122583,-2.705001
14,1.413393e+18,-0.030718,0.041888,0.094946,9.659550,-0.375922,-2.917478
16,1.413393e+18,-0.000698,0.024435,0.077493,9.291801,-0.490333,-2.598762
18,1.413393e+18,0.032812,0.008378,0.075398,9.447073,-0.098067,-2.655968


In [147]:
df['#timestamp [ns]'] = df['#timestamp [ns]'].apply(lambda x: f"{x:.0f}")

In [148]:
df.head(10)

,#timestamp [ns],w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]
0,1413393326214785280,0.094946,0.020246,0.058643,9.128357,0.106239,-2.606934
2,1413393326214785280,-0.011170,0.027925,0.069813,9.398040,1.618097,-2.705001
4,1413393326214785280,-0.092153,0.036303,0.078889,8.989429,-0.130755,-2.713173
6,1413393326214785280,-0.005585,0.009076,0.113795,9.299973,-1.234003,-2.476179
8,1413393326214785280,0.061436,0.023736,0.091455,9.275456,0.073550,-2.582418
10,1413393326214785280,0.009774,0.028623,0.060039,9.471589,0.400438,-2.606934
12,1413393326214785280,-0.032114,0.042586,0.086568,9.332662,0.122583,-2.705001
14,1413393326214785280,-0.030718,0.041888,0.094946,9.659550,-0.375922,-2.917478
16,1413393326214785280,-0.000698,0.024435,0.077493,9.291801,-0.490333,-2.598762
18,1413393326214785280,0.032812,0.008378,0.075398,9.447073,-0.098067,-2.655968


In [41]:
df.to_csv('./mav0/imu0/data_modified.csv', index=False)

In [42]:
!cp ./mav0/imu0/data_modified.csv /home/shiva/Datasets/V2_01_easy/V2_01_easy/mav0/imu0/data.csv